# 증강할 거... 메신저피싱형, 택배사칭형, 가족지인사칭형
메신저피싱형 252건 - 168건
택배사칭형 199건 - 161건
가족지인사칭형 176건 -125건

In [4]:
import pandas as pd
df= pd.read_csv("C:/Users/user/Downloads/0708/woogawooga_project/dataset/phishing_total.csv")

In [5]:
# phishing_type 별로 그룹화
#각 그룹에서 file_name 컬럼 선택
#각 그룹내에서 중복 제거한 file_name의 갯수 세기
df.groupby('phishing_type')['file_name'].nunique()

phishing_type
가족지인사칭형    125
기관사칭형      861
대출빙자형      432
메신저피싱형     168
세금환급형      103
콜백스미싱형     108
택배사칭형      161
투자권유형       91
Name: file_name, dtype: int64

In [32]:
import pandas as pd
import os
import time
import re
from dotenv import load_dotenv
from openai import OpenAI

#.env에서 OpenAI API 키 불러오기
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("❌ .env에 OPENAI_API_KEY가 설정되지 않았습니다.")

client = OpenAI(api_key=openai_api_key)

# 역번역 함수 (GPT 응답 줄 수 일치 확인 + 로그 저장)
def back_translate_with_openai(speaker_list, text_list, file_name):
    tagged_lines = [f"SPEAKER_{spk}: {txt}" for spk, txt in zip(speaker_list, text_list)]
    joined_text = "\n".join(tagged_lines)

    prompt = f"""
한국어로 된 보이스피싱 대화를 역번역해서 데이터 증강. 각 문장은 SPEAKER_0: 또는 SPEAKER_1: 으로 시작.

다음 지침을 엄격히 따르세요.
1. 각 발화를 영어로 번역한 뒤, 다시 한국어로 번역.
2. 의미는 비슷하게 유지하되 표현은 유의미하게 달라지게, 같은 문장안에서 문장 구조가 달라지게.
3. **출력 형식은 반드시 SPEAKER 태그 + 원문과 동일한 줄 수(총 {len(text_list)}줄)를 유지.**
4. 줄 수가 다를 경우 작업은 실패. 
5. **최종 출력은 반드시 한국어로만 출력합니다.**

입력:
{joined_text}

역번역 한국어 결과:
""".strip()

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.8
        )
        reply = response.choices[0].message.content

        # GPT 응답 저장
        with open("gpt_raw_log.txt", "a", encoding="utf-8") as log_file:
            log_file.write(f"\n\n===== {file_name} =====\n{reply}\n")

        # SPEAKER 태그로 분리
        matches = re.findall(r"(?i)SPEAKER[_ ]?(\d):\s*(.+)", reply)
        if len(matches) != len(text_list):
            print(f"[경고] 문장 수 불일치 - 원문: {len(text_list)}, GPT 응답: {len(matches)}")
            # 실패 로그 저장
            with open("failed_files.txt", "a", encoding="utf-8") as fail_log:
                fail_log.write(file_name + "\n")
            return [None] * len(text_list)

        return [text for _, text in matches]

    except Exception as e:
        print(f"[OpenAI API 오류] {e}")
        with open("failed_files.txt", "a", encoding="utf-8") as fail_log:
            fail_log.write(file_name + "\n")
        return [None] * len(text_list)

# 설정값
INPUT_FILE = "C:/Users/user/Downloads/0708/woogawooga_project/dataset/grouped_메신저_택배_가족지인형.csv"
OUTPUT_FILE = "cy_jgang.csv"
TARGET_TYPES = ['메신저피싱형', '택배사칭형', '가족지인사칭형']

# file_name 불러오기
def load_processed_file_names():
    if not os.path.exists(OUTPUT_FILE):
        return set()
    try:
        processed_df = pd.read_csv(OUTPUT_FILE)
        return set(processed_df['file_name'].unique())
    except:
        return set()

# 데이터 불러오기 및 필터
df = pd.read_csv(INPUT_FILE)
df = df[df['phishing_type'].isin(TARGET_TYPES)]
grouped = df.groupby("file_name")
processed_files = load_processed_file_names()

# CSV 실시간 저장
with open(OUTPUT_FILE, 'a', encoding='utf-8', newline='') as f_out:
    if os.path.getsize(OUTPUT_FILE) == 0:
        f_out.write("file_name,speaker,phishing_type,text,back_translated\n")  # 헤더

    for file_name, group in grouped:
        if file_name in processed_files:
            continue  # 이미 처리된 대화는 건너뜀

        speaker_list = group['speaker'].tolist()
        text_list = group['text'].tolist()
        phishing_type = group['phishing_type'].iloc[0]

        backtranslated_list = back_translate_with_openai(speaker_list, text_list, file_name)

        for spk, orig, back in zip(speaker_list, text_list, backtranslated_list):
            orig_safe = str(orig).replace("\n", " ").replace(",", " ")
            back_safe = str(back or "").replace("\n", " ").replace(",", " ")
            f_out.write(f"{file_name},{spk},{phishing_type},{orig_safe},{back_safe}\n")

        print(f" 처리 완료: {file_name}")
        time.sleep(1.5)



 처리 완료: phishing_1842
[경고] 문장 수 불일치 - 원문: 266, GPT 응답: 249
 처리 완료: phishing_1849
 처리 완료: phishing_486
 처리 완료: phishing_487
 처리 완료: phishing_488
 처리 완료: phishing_489
 처리 완료: phishing_490
 처리 완료: phishing_491
 처리 완료: phishing_492
 처리 완료: phishing_493
 처리 완료: phishing_494
 처리 완료: phishing_495
 처리 완료: phishing_496
 처리 완료: phishing_497
 처리 완료: phishing_498
 처리 완료: phishing_499
 처리 완료: phishing_500
 처리 완료: phishing_501
 처리 완료: phishing_502
 처리 완료: phishing_503
 처리 완료: phishing_504
 처리 완료: phishing_505
 처리 완료: phishing_506
 처리 완료: phishing_507
 처리 완료: phishing_508
 처리 완료: phishing_509
 처리 완료: phishing_510
 처리 완료: phishing_511
 처리 완료: phishing_512
 처리 완료: phishing_513
 처리 완료: phishing_514
 처리 완료: phishing_515
 처리 완료: phishing_516
 처리 완료: phishing_517
 처리 완료: phishing_518
 처리 완료: phishing_519
 처리 완료: phishing_520
 처리 완료: phishing_521
 처리 완료: phishing_522
 처리 완료: phishing_523
 처리 완료: phishing_524
 처리 완료: phishing_525
 처리 완료: phishing_526
 처리 완료: phishing_527
 처리 완료: phishing_528
 처리 완료: phishing

KeyboardInterrupt: 